In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from imblearn.over_sampling import ADASYN
from binay_load import binary_load_dataset

from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers as Layer
from tensorflow.keras.metrics import Recall, Precision, Accuracy

In [8]:
csv_loca = "star.csv"
TEST_SIZE = 0.2
VAL_SIZE = 0.2
RANDOM_STATE = 42

In [ ]:
def modelling():
  # 모델 만들기
  # 뉴런의 개수는 input과 output 사이의 숫자를 넣으라고 한다. --> 사용, 왜냐하면 시간이 적게 걸려서
  # 뉴런의 개수는 input의 2/3 정도 넣라고 한다.
  # 뉴런의 개수는 input의 두배보다는 적게 넣라고 한다.
  model = Sequential([Layer.Dense(6, input_shape)])
  model.add(Layer.Dense())

In [5]:
binary_load_dataset(csv_loca, TEST_SIZE, VAL_SIZE, RANDOM_STATE)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((14318, 8), (14318,), (3580, 8), (3580,), (3580, 8), (3580,))

In [6]:
X_train_ad.shape, X_val_ad.shape, X_test_ad.shape, y_train_ad.shape, y_val_ad.shape, y_test_ad.shape

((20711, 8), (5178, 8), (6473, 8), (20711,), (5178,), (6473,))

In [41]:
model2 = Sequential()
model2.add(Layer.Dense(256, activation='sigmoid'))
model2.add(Layer.Dense(256, activation='sigmoid'))
model2.add(Layer.Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

results = model2.fit(X_train_scaled, y_train, batch_size = 64, epochs=30, validation_data=(X_val_scaled,y_val))

Epoch 1/30
191/191 [==============================] - 2s 5ms/step - loss: 0.3126 - accuracy: 0.8988 - val_loss: 0.2532 - val_accuracy: 0.9106
Epoch 2/30
191/191 [==============================] - 1s 4ms/step - loss: 0.1843 - accuracy: 0.9326 - val_loss: 0.1256 - val_accuracy: 0.9590
Epoch 3/30
191/191 [==============================] - 1s 4ms/step - loss: 0.1147 - accuracy: 0.9620 - val_loss: 0.1237 - val_accuracy: 0.9693
Epoch 4/30
191/191 [==============================] - 1s 4ms/step - loss: 0.1015 - accuracy: 0.9681 - val_loss: 0.0980 - val_accuracy: 0.9735
Epoch 5/30
191/191 [==============================] - 1s 4ms/step - loss: 0.0932 - accuracy: 0.9714 - val_loss: 0.0965 - val_accuracy: 0.9744
Epoch 6/30
191/191 [==============================] - 1s 4ms/step - loss: 0.0931 - accuracy: 0.9719 - val_loss: 0.1028 - val_accuracy: 0.9772
Epoch 7/30
191/191 [==============================] - 1s 4ms/step - loss: 0.0921 - accuracy: 0.9727 - val_loss: 0.0949 - val_accuracy: 0.9777
Epoch 

In [42]:
model2.evaluate(X_test_scaled,  y_test, verbose=2)

112/112 - 1s - loss: 0.0772 - accuracy: 0.9785 - 509ms/epoch - 5ms/step


[0.07719853520393372, 0.9784916043281555]

In [43]:
y_test_pred = model2.predict(X_test_scaled)

y_test_pred

112/112 [==============================] - 1s 4ms/step


array([[0.00418683],
       [0.86729175],
       [0.01434747],
       ...,
       [0.00608417],
       [0.07998262],
       [0.9976125 ]], dtype=float32)

In [44]:
y_test_pred = (y_test_pred > 0.5).astype(np.float32)

y_test_pred

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [45]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_test_pred)

cm = pd.DataFrame(cm)
cm.columns = ['Predicted 0', 'Predicted 1']
cm.rename(index={0: "True 0", 1: "True 1"}, inplace = True)
cm

,Predicted 0,Predicted 1
True 0,3238,21
True 1,56,265


In [47]:
from sklearn.metrics import classification_report
print('Precision: ', round(265 / (265+21),2)) # TP / (TP+FP)
print('Recall: ', round(265 / (265+56),2)) # TP / (TP+FN)
print(classification_report(y_test, y_test_pred))

Precision:  0.93
Recall:  0.83
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      3259
           1       0.93      0.83      0.87       321

    accuracy                           0.98      3580
   macro avg       0.95      0.91      0.93      3580
weighted avg       0.98      0.98      0.98      3580



In [14]:
np.any(np.isnan(y_train))

True

In [16]:
tf.random.set_seed(7)

def model_builder(nodes=16, activation='relu'):

  model = Sequential()
  model.add(Layer.Dense(nodes, input_dim=8, activation=activation))
  model.add(Layer.Dense(nodes, input_dim=8, activation=activation))
  model.add(Layer.Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(model=model_builder, batch_size=8, verbose=0)

# GridSearch
batch_size=[64, 128, 256]
epochs=[10, 20, 30]
nodes=[64, 128, 256]
activation=['sigmoid']
param_grid=dict(batch_size=batch_size, epochs=epochs, model__nodes=nodes, model__activation=activation)


# GridSearch CV를 만들기
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [17]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.9762702951423124 using {'batch_size': 64, 'epochs': 30, 'model__activation': 'sigmoid', 'model__nodes': 256}
Means: 0.9727864326562808, Stdev: 0.0017929641355400757 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 64}
Means: 0.9725234996384672, Stdev: 0.002422339548108652 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 128}
Means: 0.972654966147374, Stdev: 0.0018800170441169062 with: {'batch_size': 64, 'epochs': 10, 'model__activation': 'sigmoid', 'model__nodes': 256}
Means: 0.9742325642542563, Stdev: 0.0017366521808483521 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 64}
Means: 0.9744297640176165, Stdev: 0.001886899374798856 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 128}
Means: 0.9725892328929205, Stdev: 0.0011270905278039916 with: {'batch_size': 64, 'epochs': 20, 'model__activation': 'sigmoid', 'model__nodes': 256}
